In [1]:
import os 
os.chdir("../")
%pwd

'd:\\Programming\\ML\\End-to-End\\CV\\End-to-End-PlantVillage'

In [2]:
from dataclasses import dataclass 
from pathlib import Path 

@dataclass(frozen=True)
class TrainingSchema:
    root_dir: Path 
    trained_model_path: Path 
    updated_base_model_path: Path 
    training_data_path: Path 
    params_epochs: int 
    params_batch_size: int 
    params_augmentation: bool 
    params_image_size: list

In [3]:
from src.PlantVillage.constants import PARAMS_FILE_PATH, CONFIG_FILE_PATH 
from src.PlantVillage.utils import create_directories, read_yaml 
import tensorflow as tf 

class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH, 
        params_filepath = PARAMS_FILE_PATH
    ):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        
        create_directories([self.config.artifacts_root])
        
        
    def get_training_config(self) -> TrainingSchema:
        training = self.config.training
        prepare_base_model = self.config.prepare_base_model 
        params = self.params 
        training_data = Path(os.path.join(self.config.data_ingestion.local_path,"PlantVillage" ))
        create_directories([training.root_dir])
        
        return TrainingSchema(
            root_dir= training.root_dir, 
            trained_model_path= training.trained_model_path, 
            updated_base_model_path= prepare_base_model.updated_base_model_path, 
            training_data_path= training_data, 
            params_epochs=params.EPOCHS, 
            params_batch_size=params.BATCH_SIZE, 
            params_augmentation= params.AUGMENTATION, 
            params_image_size= params.IMAGE_SIZE,
        )

In [6]:
class Training:
    def __init__(self, config: TrainingSchema):
        self.config = config 
        
    def get_base_model(self):
        self.model = tf.keras.models.load_model(self.config.updated_base_model_path)
        
    def train_valid_generator(self):
        
        datagenerator_kwargs = dict(
            rescale = 1./255,
            validation_split = 0.2
        ) 
        
        dataflow_kwargs = dict(
            target_size = self.config.params_image_size[:-1],
            batch_size = self.config.params_batch_size,
            interpolation = "bilinear"
        )
        

        valid_datagenerator = tf.keras.preprocessing.image.ImageDataGenerator(
            **datagenerator_kwargs
        )
        self.valid_generator = valid_datagenerator.flow_from_directory(
            directory = self.config.training_data_path,
            subset = "validation",
            shuffle = False,
            **dataflow_kwargs
        )
        
        if self.config.params_augmentation:
            train_datagenerator = tf.keras.preprocessing.image.ImageDataGenerator(
                rotation_range=40,
                horizontal_flip=True,
                width_shift_range=0.2,
                height_shift_range=0.2,
                shear_range=0.2,
                zoom_range=0.2,
                **datagenerator_kwargs
                )
        else:
            train_datagenerator = valid_datagenerator
            
        self.train_generator = train_datagenerator.flow_from_directory(
            directory = self.config.training_data_path,
            subset = "training",
            shuffle = True, 
            **dataflow_kwargs
        )
        
        
    def train(self):
        self.steps_per_epoch = self.train_generator.samples // self.train_generator.batch_size 
        self.validation_steps= self.valid_generator.samples // self.valid_generator.batch_size 
        
        self.model.fit(
            self.train_generator,
            epochs = self.config.params_epochs,
            steps_per_epoch = self.steps_per_epoch,
            validation_steps= self.validation_steps,
            validation_data = self.valid_generator
        )
        self.model.save(self.config.trained_model_path)

In [7]:
try:
    config = ConfigurationManager() 
    training_config = config.get_training_config() 
    training = Training(training_config)
    training.get_base_model() 
    training.train_valid_generator() 
    training.train()
except Exception as e:
    raise e

[2026-01-26 18:06:19,556][INFO][__init__] -> yaml file: config\config.yaml loaded Successfully.
[2026-01-26 18:06:19,559][INFO][__init__] -> yaml file: params.yaml loaded Successfully.
[2026-01-26 18:06:19,561][INFO][__init__] -> Directory created Successfully at: artifacts
[2026-01-26 18:06:19,563][INFO][__init__] -> Directory created Successfully at: artifacts/training
Found 4122 images belonging to 15 classes.
Found 16516 images belonging to 15 classes.
1032/1032 ━━━━━━━━━━━━━━━━━━━━ 1703s 2s/step - accuracy: 0.5855 - loss: 8.9472 - val_accuracy: 0.7143 - val_loss: 7.4938
